# Лекция 12: Моделирование мира с помощью ОДУ
## Практическое решение обыкновенных дифференциальных уравнений в Python

**План лекции:**
1.  Что такое ОДУ и почему они повсюду?
2.  Два подхода: аналитический и численный.
3.  **Аналитика:** Точное решение с `SymPy`.
4.  **Визуализация:** Краткий курс по `Matplotlib`.
5.  **Численные методы:** Приближенное решение с `SciPy`.
6.  **Великий трюк:** Как решить уравнение любого порядка?
7.  **Высший пилотаж:** Моделирование реальных систем.

### 1. Что такое ОДУ и почему они повсюду?

**Обыкновенное дифференциальное уравнение (ОДУ)** — это уравнение, связывающее неизвестную функцию, ее производные и независимую переменную (чаще всего — время `t`).

Проще говоря, **ОДУ описывает скорость изменения чего-либо**.

**Примеры:**
- **Физика:** Закон охлаждения Ньютона: $dT/dt = -k(T - T_{env})$. Скорость остывания тела (`dT/dt`) пропорциональна разнице его температуры и температуры окружающей среды.
- **Биология:** Рост популяции: $dP/dt = rP$. Скорость роста популяции (`dP/dt`) пропорциональна ее текущему размеру (`P`).

#### Задача Коши (Initial Value Problem)
Чтобы найти **конкретное**, единственное решение, нам нужно не только уравнение, но и **начальное условие** — состояние системы в начальный момент времени.

Задача Коши = **Дифференциальное уравнение + Начальное условие**
- `dy/dt = f(t, y)`
- `y(t₀) = y₀`

### 2. Два подхода к решению

| Подход | Цель | Инструмент в Python | Плюсы | Минусы |
| :--- | :--- | :--- | :--- | :--- |
| **Аналитический** | Найти точную формулу `y(t)` | `sympy.dsolve` | Точный, универсальный результат | Возможно только для простых уравнений |
| **Численный** | Найти значения `y` в точках `t₁, t₂, ...` | `scipy.integrate.solve_ivp` | Работает для почти любых ОДУ | Результат приближенный, есть погрешность |

### 3. Аналитика: Точное решение с `SymPy`

Давайте найдем аналитическое решение для модели радиоактивного распада: `y' = -0.5 * y`, при `y(0) = 10`.

In [ ]:
import sympy as sp

# Определяем символы: t - независимая переменная, y - функция от t
t = sp.Symbol('t')
y = sp.Function('y')(t)

# Задаем ОДУ: y' = -0.5*y
ode = sp.Eq(y.diff(t), -0.5 * y)

# Решаем ОДУ и получаем общее решение с константой C1
solution_general = sp.dsolve(ode, y)
print(f"Общее решение: {solution_general}")

# Чтобы найти C1, используем начальное условие y(0) = 10
# y(t) = C1 * exp(-0.5*t) => 10 = C1 * exp(0) => C1 = 10

# Подставляем найденное значение C1
solution_particular = solution_general.rhs.subs('C1', 10)
print(f"Частное решение: {solution_particular}")

Мы получили точную формулу: $y(t) = 10 e^{-0.5t}$. Теперь, чтобы сравнить с ней численное решение, нам нужно научиться строить графики.

### 4. Визуализация: Краткий курс по `Matplotlib`

`Matplotlib` — это стандартная библиотека для создания графиков в Python. Чтобы понять результаты решения ОДУ, без нее не обойтись.

**Краткая справка по основным командам:**
- `plt.figure(figsize=(ширина, высота))` : Создает "холст" для графика заданного размера.
- `plt.plot(x_data, y_data, label='метка')` : Рисует линию на графике. `label` используется для легенды.
- `plt.title('Название')` : Устанавливает заголовок графика.
- `plt.xlabel('Подпись оси X') / plt.ylabel('Подпись оси Y')` : Подписывает оси.
- `plt.grid(True)` : Включает сетку на графике.
- `plt.legend()` : Отображает легенду (используя `label` из `plt.plot`).
- `plt.show()` : Показывает готовый график.

Существует два основных подхода к созданию графиков:

#### Подход 1: Функциональный (на основе состояния)

Этот подход самый простой и быстрый для создания одиночных графиков. Вы используете функции напрямую из модуля `pyplot` (обычно импортируется как `plt`). Matplotlib "за кулисами" отслеживает, какой график сейчас активен, и все команды `plt.` применяются к нему.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 1. Готовим данные
x = np.linspace(0, 10, 100) # 100 точек от 0 до 10
y1 = np.sin(x)
y2 = np.cos(x)

# 2. Строим графики
plt.plot(x, y1, label='Синус') # label нужен для легенды
plt.plot(x, y2, label='Косинус')

# 3. Добавляем оформление
plt.title("Простой график")
plt.xlabel("Ось X")
plt.ylabel("Ось Y")
plt.legend() # Отображает легенду (метки из label)
plt.grid(True) # Включает сетку

# 4. Показываем результат
plt.show()

#### Подход 2: Объектно-ориентированный (ООП)

Этот подход более гибкий и мощный, он **рекомендуется** для сложных графиков или когда у вас несколько графиков в одном окне. Вы явно создаете объекты: `Figure` (весь холст) и `Axes` (область для рисования), а затем вызываете их методы.

**Главная команда:** `fig, ax = plt.subplots()`

In [ ]:
# 1. Создаем объекты Figure и Axes
# figsize=(ширина, высота) в дюймах
fig, ax = plt.subplots(figsize=(8, 5))

# Данные те же самые
x = np.linspace(0, 10, 100)
y1 = np.sin(x)
y2 = np.cos(x)

# 2. Строим графики, вызывая методы объекта ax
ax.plot(x, y1, label='Синус')
ax.plot(x, y2, label='Косинус')

# 3. Добавляем оформление через методы ax
# Обратите внимание на приставку set_*
ax.set_title("Объектно-ориентированный график")
ax.set_xlabel("Ось X")
ax.set_ylabel("Ось Y")
ax.legend()
ax.grid(True)

# 4. Показываем результат
plt.show()

#### Настройка внешнего вида (атрибуты)
Вы можете управлять почти каждым элементом графика с помощью дополнительных аргументов в функции `plot`.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

x = np.linspace(0, 10, 20) # Уменьшим число точек, чтобы маркеры были видны
y = x**2

ax.plot(
    x, y,
    color='red',          # Цвет линии
    linestyle='--',      # Стиль линии: '-' (сплошная), '--' (пунктир), ':' (точки)
    linewidth=2,          # Толщина линии
    marker='o',           # Маркеры в точках данных: 'o', 'x', 's' (квадрат)
    markersize=8,         # Размер маркеров
    label='Парабола'
)

ax.set_title("Настроенный график")
ax.legend()
ax.grid(True)
plt.show()

Теперь у вас есть все необходимые инструменты для визуализации решений ОДУ. Давайте применим их!

### 5. Численные методы: Приближенное решение с `SciPy`

Вернемся к нашей задаче: `y' = -0.5 * y`, `y(0) = 10`. Теперь решим ее численно и сравним с аналитическим решением на графике.

In [ ]:
from scipy.integrate import solve_ivp

# 1. Определяем функцию dy/dt = f(t, y)
def model(t, y):
    return -0.5 * y

# 2. Задаем начальные условия и временной интервал
y0 = [10]
t_span = [0, 10]
t_eval = np.linspace(t_span[0], t_span[1], 21) # 21 точка для наглядности

# 3. Вызываем решатель
sol = solve_ivp(model, t_span, y0, t_eval=t_eval)

# 4. Визуализируем и сравниваем
fig, ax = plt.subplots(figsize=(10, 6))

# Численное решение (точки)
ax.plot(sol.t, sol.y[0], 'o', markersize=8, label='Численное решение (solve_ivp)')

# Аналитическое решение (линия)
t_analytical = np.linspace(t_span[0], t_span[1], 200)
y_analytical = 10 * np.exp(-0.5 * t_analytical)
ax.plot(t_analytical, y_analytical, 'r-', label='Аналитическое решение')

ax.set_title("Сравнение численного и аналитического решений")
ax.set_xlabel("Время t")
ax.set_ylabel("y(t)")
ax.legend()
ax.grid(True)
plt.show()

Как видите, точки численного решения идеально ложатся на кривую аналитического. Это говорит о высокой точности численного метода.

### 6. Великий трюк: как решить уравнение ЛЮБОГО порядка?

**Проблема:** `solve_ivp` умеет решать только системы уравнений **первого** порядка. Что делать с уравнением второго порядка, например, `y'' + 3y' + 2y = 0`?

**Решение:** Превратить его в систему из двух уравнений первого порядка с помощью **замены переменных**.

**Пример:** $y'' + 3y' + 2y = 0$
Начальные условия: $y(0) = 1$, $y'(0) = 0$.

**Шаг 1: Вводим новые переменные (вектор состояния `Y`)**
- Пусть $y_1 = y$
- Пусть $y_2 = y'$

**Шаг 2: Выражаем производные новых переменных**
- $y_1' = (y)' = y' = y_2$  (Первое уравнение системы, оно всегда такое!)
- $y_2' = (y')' = y''$. Из исходного уравнения выражаем $y''$:
    - $y'' = -3y' - 2y$.
    - Заменяем $y$ и $y'$ на новые переменные: $y'' = -3y_2 - 2y_1$ (Это второе уравнение).

**Шаг 3: Записываем итоговую систему**
$$ \begin{cases} y_1' = y_2 \\ y_2' = -2y_1 - 3y_2 \end{cases} $$

In [ ]:
# Практика: Решаем ОДУ 2-го порядка

# 1. Определяем функцию для системы.
def model_2nd_order(t, y_vec):
    y1, y2 = y_vec  # y_vec = [y, y']
    dy1_dt = y2
    dy2_dt = -2 * y1 - 3 * y2
    return [dy1_dt, dy2_dt]

# 2. Задаем начальные условия для вектора [y(0), y'(0)]
y0_vec = [1, 0]
t_span = [0, 10]
t_eval = np.linspace(t_span[0], t_span[1], 101)

# 3. Вызываем решатель
sol = solve_ivp(model_2nd_order, t_span, y0_vec, t_eval=t_eval)

# 4. Визуализируем
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(sol.t, sol.y[0], label="y(t) (решение)")
ax.plot(sol.t, sol.y[1], '--', label="y'(t) (производная)")
ax.set_xlabel("Время t")
ax.set_ylabel("Значения")
ax.set_title("Решение ОДУ 2-го порядка")
ax.legend()
ax.grid(True)
plt.show()

### 7. Высший пилотаж: Моделирование реальных систем

Теперь мы можем моделировать сложные, взаимосвязанные процессы, такие как **модель "Хищник-Жертва"**.

In [ ]:
def lotka_volterra(t, z):
    x, y = z  # z = [жертвы, хищники]
    a, b, c, d = 1.5, 0.8, 1.0, 0.5
    dxdt = a * x - b * x * y
    dydt = -c * y + d * x * y
    return [dxdt, dydt]

z0 = [2, 1]
t_span = [0, 20]
t_eval = np.linspace(t_span[0], t_span[1], 1000)
sol = solve_ivp(lotka_volterra, t_span, z0, t_eval=t_eval)

# Создаем холст с двумя областями для графиков
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# График 1: Динамика популяций во времени
ax1.plot(sol.t, sol.y[0], label='Жертвы (x)')
ax1.plot(sol.t, sol.y[1], label='Хищники (y)')
ax1.set_xlabel('Время')
ax1.set_ylabel('Популяция (тыс.)')
ax1.set_title('Динамика популяций')
ax1.grid(True)
ax1.legend()

# График 2: Фазовый портрет
ax2.plot(sol.y[0], sol.y[1])
ax2.set_xlabel('Популяция жертв (x)')
ax2.set_ylabel('Популяция хищников (y)')
ax2.set_title('Фазовый портрет')
ax2.grid(True)

plt.tight_layout() # Чтобы графики не накладывались
plt.show()

### 8. Заключение

**Сегодня мы научились:**
1.  Визуализировать данные с помощью `Matplotlib`.
2.  Формулировать и решать ОДУ 1-го порядка численно и аналитически.
3.  Преобразовывать ОДУ N-го порядка в систему и решать ее.
4.  Применять эти навыки для моделирования реальных систем.

Этих знаний вам полностью хватит для выполнения итоговой лабораторной работы №12. Удачи!